In [415]:
import cv2
import mediapipe as mp
import time
import numpy as np
import pandas as pd

In [416]:
# only run this once ever per session
# try:
#     whole = pd.read_csv("new_signs_en.csv")

# except:
#     print("No dataset found")

In [417]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [418]:
cap = cv2.VideoCapture(0)
ls = []
elapsed = 0

with mp_hands.Hands(
    model_complexity = 0,
    min_detection_confidence = 0.5,
    min_tracking_confidence = 0.5
) as hands:
    
    while cap.isOpened():

        time_start = time.time()

        retval, frame = cap.read()

        if not retval:
            print("Camera Error; Exiting")
            break

        frame.flags.writeable = False
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:

                mp_drawing.draw_landmarks(
                    frame,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style()
                )

        if results.multi_hand_world_landmarks:
            for hand_landmarks in results.multi_hand_world_landmarks:
                ls.append([[lmk.x, lmk.y, lmk.z] for lmk in hand_landmarks.landmark])


        fps = 1 / (time.time() - time_start)

        frame = cv2.putText(frame, f"FPS: {round(fps)}", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)

        cv2.imshow("Hand Landmarks", frame)

        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1736981216.863267  253116 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1736981216.864736  260005 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 22.3.6), renderer: Mesa Intel(R) HD Graphics 630 (KBL GT2)
W0000 00:00:1736981216.881460  259999 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1736981216.889945  259998 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
QObject::moveToThread: Current thread (0x28f7bf0) is not the object's thread (0x2f063c0).
Cannot move to target thread (0x28f7bf0)

QObject::moveToThread: Current thread (0x28f7bf0) is not the object's thread (0x2f063c0).
Cannot move to target thread (0x28f7bf0)

QObject::moveToThread: Current thread (0x28f7bf0) is not the object's thread (0x2f063c0).
Cannot move to target

In [419]:
ls = np.array(ls)

ls

array([[[ 0.02513859,  0.05358526,  0.05666776],
        [ 0.03084178,  0.02843067,  0.03479598],
        [ 0.04508987,  0.00064396,  0.02183623],
        ...,
        [-0.02085994,  0.02181644, -0.00741172],
        [-0.0132131 ,  0.02907511, -0.01876317],
        [-0.00666727,  0.0373877 , -0.01342841]],

       [[ 0.03533898,  0.04787848,  0.04147583],
        [ 0.03836844,  0.01693057,  0.03002616],
        [ 0.04941757, -0.00815667,  0.02301195],
        ...,
        [-0.01368069,  0.01584562, -0.0052396 ],
        [-0.01027947,  0.01734254, -0.00626346],
        [-0.00836659,  0.02402288, -0.00506855]],

       [[ 0.04389426,  0.05099656,  0.01153636],
        [ 0.04651581,  0.01547328,  0.01566631],
        [ 0.05156918, -0.01155462,  0.02125984],
        ...,
        [-0.01498689,  0.01003029, -0.02812139],
        [-0.01851345, -0.00607492, -0.03972609],
        [-0.01884058, -0.02092799, -0.04535015]],

       ...,

       [[ 0.00696309,  0.07559054,  0.0078608 ],
        [ 0

In [420]:
reshaped_ls = ls.copy()
reshaped_ls = reshaped_ls.reshape(-1, 63)
reshaped_ls.shape

(534, 63)

In [421]:
df_columns = []
for i in range(21):
    df_columns.append(f"Landmark {i} x")
    df_columns.append(f"Landmark {i} y")
    df_columns.append(f"Landmark {i} z")

In [422]:
# create dataframe where rows are frames and columns are coordinates of all 21 landmarks (x, y have distinct columns)
df = pd.DataFrame(reshaped_ls, columns=df_columns)
df.shape

(534, 63)

In [423]:
# add label for handpose (DO NOT USE EMOJIS LOL)
df["Label"] = "y"

In [424]:
try:
    whole = pd.concat([whole, df], axis=0)
except:
    whole = df

whole.shape

(19619, 64)

In [425]:
whole["Label"].value_counts()

Label
d           1456
c           1416
k           1039
q            957
m            868
x            851
i            786
l            775
p            766
okay         732
r            671
o            645
f            632
g            628
n            625
u            625
t            619
a            609
w            600
not good     587
e            585
s            568
b            565
v            545
h            542
y            534
good         393
Name: count, dtype: int64

In [426]:
whole[whole["Label"] == "good"].head()

,Landmark 0 x,Landmark 0 y,Landmark 0 z,Landmark 1 x,Landmark 1 y,Landmark 1 z,Landmark 2 x,Landmark 2 y,Landmark 2 z,Landmark 3 x,...,Landmark 18 x,Landmark 18 y,Landmark 18 z,Landmark 19 x,Landmark 19 y,Landmark 19 z,Landmark 20 x,Landmark 20 y,Landmark 20 z,Label
732,-0.037530,0.025283,0.071331,-0.023538,-0.002247,0.051992,-0.021551,-0.026065,0.046599,-0.020290,...,0.003695,0.023843,-0.004019,-0.004343,0.022965,-0.001415,-0.014656,0.025710,0.000322,good
733,-0.043660,0.023239,0.063787,-0.021679,-0.005747,0.051859,-0.015757,-0.025957,0.040206,-0.013190,...,0.014027,0.024153,0.017877,0.009851,0.025378,0.035494,0.002660,0.032693,0.043769,good
734,-0.051530,0.028312,0.058247,-0.027081,0.000099,0.048187,-0.019565,-0.020041,0.039265,-0.013668,...,0.013781,0.025802,0.018954,0.008727,0.027022,0.041238,0.001672,0.033809,0.051694,good
735,-0.052828,0.031573,0.058133,-0.028352,0.002001,0.050642,-0.021189,-0.018299,0.041453,-0.012829,...,0.015558,0.027155,0.020076,0.009841,0.029779,0.042574,0.002923,0.035615,0.053516,good
736,-0.051685,0.031216,0.061570,-0.027190,0.002354,0.052111,-0.019695,-0.018768,0.042904,-0.011356,...,0.014085,0.028600,0.019067,0.008531,0.031537,0.040469,0.000791,0.036464,0.049544,good


In [427]:
# just in case of bad data
def delete_label(df, label):
    return df[df["Label"] != label]

# whole = delete_label(whole, "r")

In [429]:
whole.to_csv("new_signs_en.csv", index=False)